In [2]:
from langchain_openai import ChatOpenAI
from crewai_tools import PDFSearchTool
from crewai.tools import tool
from crewai import Crew
from crewai import Task
from crewai import Agent
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

c:\Users\Saim Safeer\Desktop\RAG\agentic_rag_ai\env\Lib\site-packages\pydantic\_internal\_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
c:\Users\Saim Safeer\Desktop\RAG\agentic_rag_ai\env\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "schema" in "DatabricksQueryToolSchema" shadows an attribute in parent "BaseModel"
  warnings.warn(
c:\Users\Saim Safeer\Desktop\RAG\agentic_rag_ai\env\Lib\site-packages\pydantic\_internal\_generate_schema.py:617: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the typ

In [4]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [5]:
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=1,
    max_tokens=1000
)

In [17]:
response = llm.invoke("What is the capital of Pakistan?")
print(response)

content='The capital of Pakistan is Islamabad.' response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.062000846, 'prompt_time': 0.002014243, 'completion_time': 0.029090909, 'total_time': 0.031105152}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_90c1d253ff', 'finish_reason': 'stop', 'logprobs': None} id='run-4f9df476-46dc-41bb-9f12-253601150fb0-0'


In [3]:
# loading pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

Name: langchain-community
Version: 0.3.20
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: C:\Users\Saim Safeer\Desktop\RAG\agentic_rag_ai\env\Lib\site-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: embedchain, langchain-experimental
